In [3]:
# 랜덤 포레스트
import pandas as pd
df = pd.read_csv('data/my/heart_2020_cleaned.csv')
# pd.set_option('future.no_silent_downcasting', True)
# df = df.replace({'No': 0, 'Yes': 1}) # 일괄적으로 no=0, yes=1로 변환
# no=1, yes=1로 일괄 변경하려던 눈물나는 흔적..
        # 맨 처음에 모든 컬럼을 변수로 잡고 분석을 해보려고 했는데.. 그래서 no=0, yes=1으로 일괄 바꿔주고, 나머지 문자열은 분석에서 제외하려 했는데... object인 탓에 변경을 하려면 한세월일 것 같아서 잽싸게 포기했다...!
            # 그래서 애초에 float인 컬럼 4개만 x에 할당하고 분석을 진행했다
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [4]:
# 불러오기
y = df['HeartDisease'].map({'No': 0, 'Yes': 1})
x = df[['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']]

# 셋 구분
# x = input, y=target
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)

# 클래스 선언 + 교차 검증\n",
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
rf = RandomForestClassifier()
scores = cross_validate(rf, x_train, y_train, return_train_score=True)
print(scores)
    # 과대적합 + 속도 너무 오래 걸림: 19

{'fit_time': array([19.93349171, 15.72529793, 20.42244387, 19.5484252 , 22.11352205]), 'score_time': array([1.13975573, 1.16606688, 1.14881492, 1.08568907, 1.18812323]), 'test_score': array([0.89528872, 0.89536993, 0.89474452, 0.89549501, 0.89470283]), 'train_score': array([0.95129146, 0.95113015, 0.95157835, 0.95104155, 0.95169301])}


In [5]:
# 성능 향상 위한 여정...엑스트라 트리로
# 학습
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier()
scores2 = cross_validate(et, x_train, y_train, return_train_score=True)
print(scores2)
    # 여전히 속도 너무 느림...: 11
    # 그리고 과대적합 문제가 여전히 심각하다"

{'fit_time': array([11.67372632, 11.60630441, 10.96128273, 11.51878262, 11.49808073]), 'score_time': array([1.45673347, 1.32353902, 1.34691787, 1.42529964, 1.43441439]), 'test_score': array([0.89013967, 0.8900123 , 0.88919927, 0.89132565, 0.88949113]), 'train_score': array([0.95134358, 0.95118227, 0.95162526, 0.95107803, 0.95170865])}


In [6]:

# 성능 향상을 위한 여정2... GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
scores3 = cross_validate(gb, x_train, y_train, return_train_score=True)
print(scores3)
    # 9초... 2배나 절약했다! 장족의 발전이지만 성능 향상을 위한 여정에 다시 떠나본다
    # 그래도 그레이디언트부스팅을 하니까, 과대적합은 많이 해소됐다!"

{'fit_time': array([10.6576705 , 11.00800633,  9.62571812,  9.62566137, 10.03331017]), 'score_time': array([0.05931878, 0.06103086, 0.0819149 , 0.06255436, 0.0637784 ]), 'test_score': array([0.91432145, 0.91436136, 0.91429882, 0.91431966, 0.91434051]), 'train_score': array([0.91440826, 0.91441392, 0.91440871, 0.91438265, 0.91441392])}


In [7]:
# 성능 향상을 위한 여정3... HistGradientBoostingClassifier.. 이름도 참 길다, 속도는 빠르길..
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier()
scores4 = cross_validate(hgb, x_train, y_train, return_train_score=True)
print(scores4)
# 댑박!! 0.8초로 엄청 단축됐다! 이름은 엄청 길지만 속도는 초특급이었다~
# 과대적합 역시 보이지 않고, train과 test score가 비슷한 수준으로 긍정적이다!"

{'fit_time': array([1.09479427, 0.87837434, 0.94323945, 0.95362425, 0.84527898]), 'score_time': array([0.09726691, 0.11243367, 0.12107849, 0.12233233, 0.12283444]), 'test_score': array([0.9143423 , 0.91436136, 0.91436136, 0.91436136, 0.91434051]), 'train_score': array([0.91435615, 0.91435138, 0.91435138, 0.91435138, 0.91435659])}


In [9]:
# 클러스터
# 불러오기
import numpy as np
fruits = np.load('data/fruits_300.npy')
fruits_2d = fruits.reshape(-1, 100*100)

# 학습
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3) # n_clusters: 몇개의 과일을 분류할지 정하는 것
km.fit(fruits_2d)
km.labels_
# 클러스터는 시각화가 핵심일 것 같은데 그건 더 연습을 해봐야 할 듯,,

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], d